Install Pandas

In [1]:
!pip3 install pandas python-highcharts python-dateutil


In [2]:
import pandas
# load the TSV log of group and private chats. Vis-a-vis chat has the group name which is the other contact
convs = pandas.read_csv('/Users/utente/IdeaProjects/IngestAdiumLogs/adium_chat_messages.tsv.bkp', sep='\t', names=['group', 'ts', 'user', 'text'])



In [3]:
# a group has an ID and a name, e.g. $27364523746234_my friends
# the ID is constant, the name changes, keep the first name seen for an ID to make it more user-friendly
friendly_names = {}
def grouper_fun(ind):
    name = convs['group'].loc[ind]
    if name.split('_')[0] in friendly_names:
        return friendly_names[name.split('_')[0]]
    else:
        friendly_names[name.split('_')[0]] = name
        return name
# limit the timestamp to the hour
def timestamper_fun(ind):
    return convs['ts'].loc[ind][:10]

grouped = convs.groupby(by=timestamper_fun)

In [4]:
counters = grouped.count() #.sort_values(by=['index'], ascending=False)
#list(grouped.count())


In [5]:
from highcharts import Highchart


# A chart is the container that your data will be rendered in, it can (obviously) support multiple data series within it.
chart = Highchart()

date_labels = counters.index
xy_couples = list(map(lambda x: [str(x[0]), int(x[1][0])],
                      zip(date_labels, counters.values)
                     ))

import dateutil.parser
for i in range(len(xy_couples)):
    xy_couples[i][0] = dateutil.parser.parse(xy_couples[i][0])
    
#print(xy_couples)
    
# Adding a series requires at minimum an array of data points. 
# You can also change the series type, the name, or other series options as kwargs.
chart.add_data_set(xy_couples, series_type='line', name='Example Series')

# This will generate and save a .html file at the location you assign
#chart.save_file()
# just embed the chart in the notebook



chart.set_options('chart', {'inverted': False})

options = {
    'title': {
        'text': 'Messages per day'
    },
    'subtitle': {
        'text': 'Messages in all groups for each hour in the timeseries'
    },
    'xAxis': {
        'reversed': False,
        'title': {
            'enabled': True,
            'text': 'time'
        },
        'labels': {
            'formatter': 'function () {\
                return new Date(this.value).toISOString();\
            }'
        },
        'maxPadding': 0.05,
        'showLastLabel': True
    },
    'yAxis': {
        'title': {
            'text': 'Messages'
        },
        'labels': {
            'formatter': "function () {\
                return this.value;\
            }"
        },
        'lineWidth': 1
    },
    'legend': {
        'enabled': False
    },
    'tooltip': {
        'headerFormat': '<b>{series.name}</b><br/>',
         'formatter': "function () {\
                return new Date(this.x).toISOString() + ' : ' + this.y;\
            }"
    }
}

chart.set_dict_options(options)

chart.add_data_set(xy_couples, 'spline', 'Messages', marker={'enabled': False})

chart